In [523]:
import json
import pandas as pd
from datetime import datetime
from listas import priorfalls_list, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [524]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,albumina,proteinas totales,Hb,colesterol total,leucos,limfos,prot C react,urea,FGE MDRD,FGE CDK-EPI
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,None,None,None,None,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,3.1,5.3,14.5,124.7,X,38.9,19.61,25.7,>60,>90
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,None,None,None,None,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,None,None,None,None,None,None
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,X,None,X,X,0.54,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...,Proves afegides a un altre petició per extracc...


In [525]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA_MECVV = tabla[
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) |
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0)]   
SIN_PA = tabla[
          (tabla["P_diagnosticada"] == 1.0) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30)]

# Síndromes geriàtrics

## Index de Charslton

## Caigudes prèvies

In [526]:
caprev_CON_PA= (CON_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_CON_PA:.2f}%")

Percentatge de caidas previas: 5.88%


In [527]:
caprev_CON_PA_MECVV= (CON_PA_MECVV['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_CON_PA_MECVV:.2f}%")

Percentatge de caidas previas: 0.00%


In [528]:
caprev_SIN_PA= (SIN_PA['caidas_previas'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SIN_PA:.2f}%")

Percentatge de caidas previas: 6.12%


#### Pvalor

##### Normalitat

In [529]:
stat, p_value = shapiro(CON_PA['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [530]:
stat, p_value = shapiro(CON_PA_MECVV['caidas_previas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [531]:
stat, p_value = shapiro(SIN_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [532]:
stat, p_value_12 = mannwhitneyu(CON_PA['caidas_previas'], CON_PA_MECVV['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.49002535950179893


In [533]:
stat, p_value_13 = mannwhitneyu(CON_PA['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.9858477456716862


In [534]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['caidas_previas'], SIN_PA['caidas_previas'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.44143809205435813


## Deliris

In [535]:
del_CON_PA= (CON_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_CON_PA:.2f}%")

Percentatge de delirios: 23.53%


In [536]:
del_CON_PA_MECVV= (CON_PA_MECVV['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_CON_PA_MECVV:.2f}%")

Percentatge de delirios: 40.00%


In [537]:
del_SIN_PA= (SIN_PA['delirios'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SIN_PA:.2f}%")

Percentatge de delirios: 18.37%


#### Pvalor

##### Normalitat

In [538]:
stat, p_value = shapiro(CON_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [539]:
stat, p_value = shapiro(CON_PA_MECVV['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.6405
Valor p = 0.0002
El número de filas no sigue una distribución normal (rechazar H0).


In [540]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [541]:
stat, p_value_12 = mannwhitneyu(CON_PA['delirios'], CON_PA_MECVV['delirios'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.39174655228054167


In [542]:
stat, p_value_13 = mannwhitneyu(CON_PA['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.6549000779898075


In [543]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['delirios'], SIN_PA['delirios'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.13960802912048415


## Demència

In [544]:
dem_CON_PA= (CON_PA['demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_CON_PA:.2f}%")

Percentatge de demencia: 47.06%


In [545]:
dem_CON_PA_MECVV= (CON_PA_MECVV['demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_CON_PA_MECVV:.2f}%")

Percentatge de demencia: 40.00%


In [546]:
dem_SIN_PA= (SIN_PA['demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_SIN_PA:.2f}%")

Percentatge de demencia: 30.61%


#### Pvalor

##### Normalitat

In [547]:
stat, p_value = shapiro(CON_PA['demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.6418
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [548]:
stat, p_value = shapiro(CON_PA_MECVV['demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.6405
Valor p = 0.0002
El número de filas no sigue una distribución normal (rechazar H0).


In [549]:
stat, p_value = shapiro(SIN_PA['delirios'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.4716
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [550]:
stat, p_value_12 = mannwhitneyu(CON_PA['demència'], CON_PA_MECVV['demència'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.7484826918143946


In [551]:
stat, p_value_13 = mannwhitneyu(CON_PA['demència'], SIN_PA['demència'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.22703869678020527


In [552]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['demència'], SIN_PA['demència'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.5743933608939569


## Síndrome depressiu

In [553]:
depre_CON_PA= (CON_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_CON_PA:.2f}%")

Percentatge de sindrome depresivo: 23.53%


In [554]:
depre_CON_PA_MECVV= (CON_PA_MECVV['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_CON_PA_MECVV:.2f}%")

Percentatge de sindrome depresivo: 0.00%


In [555]:
depre_SIN_PA= (SIN_PA['sindrome_depresivo'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SIN_PA:.2f}%")

Percentatge de sindrome depresivo: 14.29%


#### Pvalor

##### Normalitat

In [556]:
stat, p_value = shapiro(CON_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [557]:
stat, p_value = shapiro(CON_PA_MECVV['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [558]:
stat, p_value = shapiro(SIN_PA['sindrome_depresivo'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [559]:
stat, p_value_12 = mannwhitneyu(CON_PA['sindrome_depresivo'], CON_PA_MECVV['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.11180527770715322


In [560]:
stat, p_value_13 = mannwhitneyu(CON_PA['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.3880726918610017


In [561]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sindrome_depresivo'], SIN_PA['sindrome_depresivo'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.21340049774345338


## Incontinència urinària

In [562]:
ic_CON_PA= (CON_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_CON_PA:.2f}%")

Percentatge de incontinencia urinaria: 29.41%


In [563]:
ic_CON_PA_MECVV= (CON_PA_MECVV['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_CON_PA_MECVV:.2f}%")

Percentatge de incontinencia urinaria: 30.00%


In [564]:
ic_SIN_PA= (SIN_PA['incont_uri'] == 1.0).mean() * 100
print(f"Percentatge de incontinència urinària: {ic_SIN_PA:.2f}%")

Percentatge de incontinencia urinaria: 14.29%


#### Pvalor

##### Normalitat

In [565]:
stat, p_value = shapiro(CON_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5785
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [566]:
stat, p_value = shapiro(CON_PA_MECVV['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5942
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [567]:
stat, p_value = shapiro(SIN_PA['incont_uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [568]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_uri'], CON_PA_MECVV['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 1.0


In [569]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.17015925929193942


In [570]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_uri'], SIN_PA['incont_uri'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.237514563226218


## Incontinència fecal 

In [571]:
ifec_CON_PA= (CON_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_CON_PA:.2f}%")

Percentatge de incontinencia fecal: 29.41%


In [572]:
ifec_CON_PA_MECVV= (CON_PA_MECVV['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_CON_PA_MECVV:.2f}%")

Percentatge de incontinencia fecal: 10.00%


In [573]:
ifec_SIN_PA= (SIN_PA['incont_fec'] == 1.0).mean() * 100
print(f"Percentatge de incontinència fecal: {ifec_SIN_PA:.2f}%")

Percentatge de incontinencia fecal: 12.24%


#### Pvalor

##### Normalitat

In [574]:
stat, p_value = shapiro(CON_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5785
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [575]:
stat, p_value = shapiro(CON_PA_MECVV['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [576]:
stat, p_value = shapiro(SIN_PA['incont_fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3844
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [577]:
stat, p_value_12 = mannwhitneyu(CON_PA['incont_fec'], CON_PA_MECVV['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.2649031747093591


In [578]:
stat, p_value_13 = mannwhitneyu(CON_PA['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.10681676423525503


In [579]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['incont_fec'], SIN_PA['incont_fec'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.8568943909879498


## Úlceres de pressió

In [580]:
up_CON_PA= (CON_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_CON_PA:.2f}%")

Percentatge de ulceras de presion: 5.88%


In [581]:
up_CON_PA_MECVV= (CON_PA_MECVV['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_CON_PA_MECVV:.2f}%")

Percentatge de ulceras de presion: 0.00%


In [582]:
up_SIN_PA= (SIN_PA['ulceras_presion'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SIN_PA:.2f}%")

Percentatge de ulceras de presion: 2.04%


#### Pvalor

##### Normalitat

In [583]:
stat, p_value = shapiro(CON_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [584]:
stat, p_value = shapiro(CON_PA_MECVV['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [585]:
stat, p_value = shapiro(SIN_PA['ulceras_presion'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.1273
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [586]:
stat, p_value_12 = mannwhitneyu(CON_PA['ulceras_presion'], CON_PA_MECVV['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.49002535950179893


In [587]:
stat, p_value_13 = mannwhitneyu(CON_PA['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.4440308448923743


In [588]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['ulceras_presion'], SIN_PA['ulceras_presion'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6843175494727258


## Immobilitat

## Confusió

## Osteoporosi

In [589]:
ost_CON_PA= (CON_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_CON_PA:.2f}%")

Percentatge de osteoporosis: 0.00%


In [590]:
ost_CON_PA_MECVV= (CON_PA_MECVV['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_CON_PA_MECVV:.2f}%")

Percentatge de osteoporosis: 0.00%


In [591]:
ost_SIN_PA= (SIN_PA['osteoporosis'] == 1.0).mean() * 100
print(f"Percentatge de osteoporosi: {ost_SIN_PA:.2f}%")

Percentatge de osteoporosis: 0.00%


#### Pvalor

##### Normalitat

In [592]:
stat, p_value = shapiro(CON_PA['osteoporosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [593]:
stat, p_value_12 = mannwhitneyu(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 1.0


In [594]:
stat, p_value_12 = ttest_ind(CON_PA['osteoporosis'], CON_PA_MECVV['osteoporosis'], equal_var= False)
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV nan


## Sarcopènia

In [595]:
sarc_CON_PA= (CON_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_CON_PA:.2f}%")

Percentatge de sarcopenia: 0.00%


In [596]:
sarc_CON_PA_MECVV= (CON_PA_MECVV['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_CON_PA_MECVV:.2f}%")

Percentatge de sarcopenia: 10.00%


In [597]:
sarc_SIN_PA= (SIN_PA['sarcopenia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SIN_PA:.2f}%")

Percentatge de sarcopenia: 4.08%


#### Pvalor

##### Normalitat

In [598]:
stat, p_value = shapiro(CON_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [599]:
stat, p_value = shapiro(CON_PA_MECVV['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [600]:
stat, p_value = shapiro(SIN_PA['sarcopenia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2008
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [601]:
stat, p_value_12 = mannwhitneyu(CON_PA['sarcopenia'], CON_PA_MECVV['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.21976847950216105


In [602]:
stat, p_value_13 = mannwhitneyu(CON_PA['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.41519527301502746


In [603]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['sarcopenia'], SIN_PA['sarcopenia'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.45734646700671056


## Trastorns del son

In [604]:
ts_CON_PA= (CON_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_CON_PA:.2f}%")

Percentatge de trastornos del sueño: 0.00%


In [605]:
ts_CON_PA_MECVV= (CON_PA_MECVV['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_CON_PA_MECVV:.2f}%")

Percentatge de trastornos del sueño: 0.00%


In [606]:
ts_SIN_PA= (SIN_PA['problsueño'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SIN_PA:.2f}%")

Percentatge de trastornos del sueño: 0.00%


## Dolor crònic

In [607]:
dc_CON_PA= (CON_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_CON_PA:.2f}%")

Percentatge de dolor cronico: 0.00%


In [608]:
dc_CON_PA_MECVV= (CON_PA_MECVV['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_CON_PA_MECVV:.2f}%")

Percentatge de dolor cronico: 0.00%


In [609]:
dc_SIN_PA= (SIN_PA['dolor_cron'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SIN_PA:.2f}%")

Percentatge de dolor cronico: 0.00%


## Iatrogènic

In [610]:
iatr_CON_PA= (CON_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_CON_PA:.2f}%")

Percentatge de iatrogenico: 0.00%


In [611]:
iatr_CON_PA_MECVV= (CON_PA_MECVV['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_CON_PA_MECVV:.2f}%")

Percentatge de iatrogenico: 0.00%


In [612]:
iatr_SIN_PA= (SIN_PA['iatrogenico'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SIN_PA:.2f}%")

Percentatge de iatrogenico: 0.00%


## Restrenyiment

In [613]:
est_CON_PA= (CON_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_CON_PA:.2f}%")

Percentatge de estreñimiento: 0.00%


In [614]:
est_CON_PA_MECVV= (CON_PA_MECVV['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_CON_PA_MECVV:.2f}%")

Percentatge de estreñimiento: 0.00%


In [615]:
est_SIN_PA= (SIN_PA['estreñimiento'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SIN_PA:.2f}%")

Percentatge de estreñimiento: 0.00%


# Historial mèdic/comorbilitats

## Malalties CV

In [616]:
cv_CON_PA= (CON_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_CON_PA:.2f}%")

Percentatge de problemas CV: 5.88%


In [617]:
cv_CON_PA_MECVV= (CON_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_CON_PA_MECVV:.2f}%")

Percentatge de problemas CV: 10.00%


In [618]:
cv_SIN_PA= (SIN_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SIN_PA:.2f}%")

Percentatge de problemas CV: 6.12%


#### Pvalor

##### Normalitat 

In [619]:
stat, p_value = shapiro(CON_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2622
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [620]:
stat, p_value = shapiro(CON_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [621]:
stat, p_value = shapiro(SIN_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2580
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [622]:
stat, p_value_12 = mannwhitneyu(CON_PA['CV'], CON_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.7400102360583588


In [623]:
stat, p_value_13 = mannwhitneyu(CON_PA['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.9858477456716862


In [624]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6762982127217047


## Malalties del cor

In [625]:
cora_CON_PA= (CON_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_CON_PA:.2f}%")

Percentatge de enfermedades del corazon: 11.76%


In [626]:
cora_CON_PA_MECVV= (CON_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_CON_PA_MECVV:.2f}%")

Percentatge de enfermedades del corazon: 10.00%


In [627]:
cora_SIN_PA= (SIN_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_SIN_PA:.2f}%")

Percentatge de enfermedades del corazon: 14.29%


#### Pvalor

##### Normalitat

In [628]:
stat, p_value = shapiro(CON_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3853
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [629]:
stat, p_value = shapiro(CON_PA_MECVV['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [630]:
stat, p_value = shapiro(SIN_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.4168
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [631]:
stat, p_value_12 = mannwhitneyu(CON_PA['probl_corazon'], CON_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.9265568396984611


In [632]:
stat, p_value_13 = mannwhitneyu(CON_PA['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.9858477456716862


In [633]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['CV'], SIN_PA['CV'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.6762982127217047


## Enfermedades neurodegenerativas

In [634]:
nd_CON_PA= (CON_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de enfermedades neurodegenerativas: {nd_CON_PA:.2f}%")

Percentatge de enfermedades neurodegenerativas: 0.00%


In [635]:
nd_CON_PA_MECVV= (CON_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de enfermedades neurodegenerativas: {nd_CON_PA_MECVV:.2f}%")

Percentatge de enfermedades neurodegenerativas: 10.00%


In [636]:
nd_SIN_PA= (SIN_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de enfermedades neurodegenerativas: {nd_SIN_PA:.2f}%")

Percentatge de enfermedades neurodegenerativas: 4.08%


#### Pvalor

##### Normalitat

In [637]:
stat, p_value = shapiro(CON_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [638]:
stat, p_value = shapiro(CON_PA_MECVV['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [639]:
stat, p_value = shapiro(SIN_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2008
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [640]:
stat, p_value_12 = mannwhitneyu(CON_PA['neurodegenerativas'], CON_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.21976847950216105


In [641]:
stat, p_value_13 = mannwhitneyu(CON_PA['neurodegenerativas'], SIN_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.41519527301502746


In [642]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['neurodegenerativas'], SIN_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.45734646700671056


## Diabetes mellitus

In [643]:
dm_CON_PA= (CON_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_CON_PA:.2f}%")

Percentatge de diabetes mellitus: 23.53%


In [644]:
dm_CON_PA_MECVV= (CON_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_CON_PA_MECVV:.2f}%")

Percentatge de diabetes mellitus: 10.00%


In [645]:
dm_SIN_PA= (SIN_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_SIN_PA:.2f}%")

Percentatge de diabetes mellitus: 26.53%


#### Pvalor 

##### Normalitat

In [646]:
stat, p_value = shapiro(CON_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [647]:
stat, p_value = shapiro(CON_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [648]:
stat, p_value = shapiro(SIN_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5514
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [649]:
stat, p_value_12 = mannwhitneyu(CON_PA['DM'], CON_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.4120280546315189


In [650]:
stat, p_value_13 = mannwhitneyu(CON_PA['DM'], SIN_PA['DM'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PAN", p_value_13)

P-valor entre CON_PA i SIN_PAN 0.8163159339942008


In [651]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['DM'], SIN_PA['DM'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PAN", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PAN 0.2728301710651724


## Hepatopatias

In [652]:
hp_CON_PA= (CON_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopatias: {hp_CON_PA:.2f}%")

Percentatge de hepatopatias: 11.76%


In [653]:
hp_CON_PA_MECVV= (CON_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopatias: {hp_CON_PA_MECVV:.2f}%")

Percentatge de hepatopatias: 10.00%


In [654]:
hp_SIN_PA= (SIN_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopatias: {hp_SIN_PA:.2f}%")

Percentatge de hepatopatias: 6.12%


#### Pvalor

##### Normalitat

In [655]:
stat, p_value = shapiro(CON_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3853
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [ ]:
stat, p_value = shapiro(CON_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

In [450]:
stat, p_value = shapiro(SIN_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2580
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [451]:
stat, p_value_12 = mannwhitneyu(CON_PA['hepatopatias'], CON_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.9265568396984611


In [452]:
stat, p_value_13 = mannwhitneyu(CON_PA['hepatopatias'], SIN_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.4618706673466527


In [453]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['hepatopatias'], SIN_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.6762982127217047


## Neoplasia

In [454]:
neopl_CON_PA= (CON_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplasia: {neopl_CON_PA:.2f}%")

Porcentaje de neoplasia: 0.00%


In [455]:
neopl_CON_PA_MECVV= (CON_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplasia: {neopl_CON_PA_MECVV:.2f}%")

Porcentaje de neoplasia: 0.00%


In [456]:
neopl_SIN_PA= (SIN_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplasia: {neopl_SIN_PA:.2f}%")

Porcentaje de neoplasia: 6.12%


#### Pvalor

##### Normalitat

In [457]:
stat, p_value = shapiro(CON_PA['neoplasias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


In [458]:
stat, p_value = shapiro(CON_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.3657
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [459]:
stat, p_value = shapiro(SIN_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.2580
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [460]:
stat, p_value_12 = mannwhitneyu(CON_PA['neoplasias'], CON_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 1.0


In [461]:
stat, p_value_13 = mannwhitneyu(CON_PA['neoplasias'], SIN_PA['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_12)

P-valor entre CON_PA y SIN_PA 1.0


In [462]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['neoplasias'], SIN_PA['neoplasias'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.44143809205435813


## Enfermedad cronica renal (creatinina > 1.5mg/dL)

Dado que esta columna es de tipo objeto primero la pasaré a tipo numerico para poder operar con ella. 

In [463]:
CON_PA.loc[:, 'creatinina'] = pd.to_numeric(CON_PA['creatinina'], errors='coerce')
CON_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(CON_PA_MECVV['creatinina'], errors='coerce')
SIN_PA.loc[:, 'creatinina'] = pd.to_numeric(SIN_PA['creatinina'], errors='coerce')

In [464]:
ecr_CON_PA= (CON_PA['creatinina'] >= 1.5).mean() * 100
print(f"Percentatge de enfermedad cronica renal con creatinina >1.5: {ecr_CON_PA:.2f}%")

Porcentaje de enfermedad cronica renal con creatinina >1.5: 0.00%


In [465]:
ecr_CON_PA_MECVV= (CON_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de enfermedad cronica renal con creatinina >1.5: {ecr_CON_PA_MECVV:.2f}%")

Porcentaje de enfermedad cronica renal con creatinina >1.5: 0.00%


In [466]:
ecr_SIN_PA= (SIN_PA['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de enfermedad cronica renal con creatinina >1.5: {ecr_SIN_PA:.2f}%")

Porcentaje de enfermedad cronica renal con creatinina >1.5: 6.12%


#### Pvalor

##### Normalitat

In [467]:
stat, p_value = shapiro(CON_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.9549
Valor p = 0.7439
El número de filas sigue una distribución normal (aceptar H0).


In [468]:
stat, p_value = shapiro(CON_PA_MECVV['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.9815
Valor p = 0.9110
El número de filas sigue una distribución normal (aceptar H0).


In [469]:
stat, p_value = shapiro(SIN_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.7235
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [470]:
stat, p_value_12 = ttest_ind(CON_PA['creatinina'], CON_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV nan


In [471]:
stat, p_value_13 = mannwhitneyu(CON_PA['creatinina'], SIN_PA['creatinina'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA nan


In [472]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['creatinina'], SIN_PA['creatinina'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA nan


## Fallo renal agudo (o insuficiencia renal aguda)

In [473]:
ira_CON_PA= (CON_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallo renal agudo: {ira_CON_PA:.2f}%")

Porcentaje de fallo renal agudo: 23.53%


In [474]:
ira_CON_PA_MECVV= (CON_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallo renal agudo: {ira_CON_PA_MECVV:.2f}%")

Porcentaje de fallo renal agudo: 50.00%


In [475]:
ira_SIN_PA= (SIN_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallo renal agudo: {ira_SIN_PA:.2f}%")

Porcentaje de fallo renal agudo: 26.53%


#### Pvalor

##### Normalitat

In [493]:
stat, p_value = shapiro(CON_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5327
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [481]:
stat, p_value = shapiro(CON_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.6553
Valor p = 0.0003
El número de filas no sigue una distribución normal (rechazar H0).


In [482]:
stat, p_value = shapiro(SIN_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.5514
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [497]:
stat, p_value_12 = mannwhitneyu(CON_PA['ARF'], CON_PA_MECVV['ARF'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 0.17639454522781728


In [498]:
stat, p_value_13 = mannwhitneyu(CON_PA['ARF'], SIN_PA['ARF'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.8163159339942008


In [500]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['ARF'], SIN_PA['ARF'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_23)

P-valor entre CON_PA y SIN_PA 0.14880872192595962


## Mareos/sincope/lipotimia/reaccion vasovagal

In [489]:
dizsyn_CON_PA= (CON_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de mareos/sincope/lipotimia/reaccion vasovagal: {dizsyn_CON_PA:.2f}%")

Porcentaje de mareos/sincope/lipotimia/reaccion vasovagal: 0.00%


In [490]:
dizsyn_CON_PA_MECVV= (CON_PA_MECVV['mareos'] == 1.0).mean() * 100
print(f"Percentatge de mareos/sincope/lipotimia/reaccion vasovagal: {dizsyn_CON_PA_MECVV:.2f}%")

Porcentaje de mareos/sincope/lipotimia/reaccion vasovagal: 0.00%


In [491]:
dizsyn_SIN_PA= (SIN_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de mareos/sincope/lipotimia/reaccion vasovagal: {dizsyn_SIN_PA:.2f}%")

Porcentaje de mareos/sincope/lipotimia/reaccion vasovagal: 2.04%


#### Pvalor

##### Normalitat

In [494]:
stat, p_value = shapiro(CON_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [495]:
stat, p_value = shapiro(CON_PA_MECVV['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [496]:
stat, p_value = shapiro(SIN_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 0.1273
Valor p = 0.0000
El número de filas no sigue una distribución normal (rechazar H0).


In [502]:
stat, p_value_12 = ttest_ind(CON_PA['mareos'], CON_PA_MECVV['mareos'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV nan


In [505]:
stat, p_value_12 = mannwhitneyu(CON_PA['mareos'], CON_PA_MECVV['mareos'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA y CON_PA_MECVV 1.0


In [503]:
stat, p_value_13 = mannwhitneyu(CON_PA['mareos'], SIN_PA['mareos'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA y SIN_PA 0.5793276078637376


In [504]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['mareos'], SIN_PA['mareos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV y SIN_PA 0.6843175494727258


## VIH

In [507]:
vih_CON_PA= (CON_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_CON_PA:.2f}%")

Porcentaje de VIH: 0.00%


In [508]:
vih_CON_PA_MECVV= (CON_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_CON_PA_MECVV:.2f}%")

Porcentaje de VIH: 0.00%


In [509]:
vih_SIN_PA= (SIN_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SIN_PA:.2f}%")

Porcentaje de VIH: 0.00%


#### Pvalor

##### Normalitat

In [511]:
stat, p_value = shapiro(CON_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [506]:
stat, p_value = shapiro(CON_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

In [512]:
stat, p_value = shapiro(SIN_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístico W = 1.0000
Valor p = 1.0000
El número de filas sigue una distribución normal (aceptar H0).


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


## Psicosi

In [513]:
psi_CON_PA= (CON_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_CON_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [516]:
psi_CON_PA_MECVV= (CON_PA_MECVV['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_CON_PA_MECVV:.2f}%")

In [518]:
psi_SIN_PA= (SIN_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SIN_PA:.2f}%")

Percentatge de psicosi: 0.00%


## Deficiències nutricionals

In [520]:
nutri_CON_PA= (CON_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiència nutricional: {nutri_CON_PA:.2f}%")

Percentatge de deficiència nutricional: 0.00%


In [521]:
nutri_CON_PA_MECVV= (CON_PA_MECVV['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiència nutricional: {nutri_CON_PA_MECVV:.2f}%")

Percentatge de deficiència nutricional: 0.00%


In [522]:
nutri_SIN_PA= (SIN_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiència nutricional: {nutri_SIN_PA:.2f}%")

Percentatge de deficiència nutricional: 0.00%
